In [ ]:
import cv2
import pandas as pd
import numpy as np
import time
import tensorflow as tf
import zipfile
import matplotlib.pyplot as plt
import os
%matplotlib inline

from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file = zipfile.ZipFile(file = "/content/drive/MyDrive/Deep_Learning/Face_Recognation/Material.zip",mode = "r")
zip_file.extractall("./")
zip_file.close()

In [ ]:
directory = "/content/drive/MyDrive/Deep_Learning/Face_Recognation/Data/"

model = tf.keras.models.load_model(os.path.join(directory,"model_01_expressions.h5"))

In [ ]:
video_path = "/content/Material/Videos/video_teste04.mp4"
cap = cv2.VideoCapture(video_path)
conn,frame = cap.read()
print(frame.shape)

(360, 640, 3)


In [ ]:
resize = True

width_max = 600

if (resize and frame.shape[1] > width_max):
    proportionate = frame.shape[1] / frame.shape[0]

    frame_width = width_max
    frame_height = int(frame_width / proportionate)

else:
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

print(frame.shape,frame_width,frame_height)

(360, 640, 3) 600 337


In [ ]:
file_name = "/content/drive/MyDrive/Deep_Learning/Face_Recognation/Videos"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
fps = 24
video_output = cv2.VideoWriter(file_name,fourcc,fps,(frame_width,frame_height))

face_detection_path = "/content/Material/haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(face_detection_path)

small_font,medium_font= 0.4, 0.7
font = cv2.FONT_HERSHEY_SIMPLEX

expressions = ['Anger', 'Disgusted', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral']

while (cv2.waitKey(1) < 0):
    conn,frame = cap.read()

    if not conn:
        break

    t = time.time()

    if resize:
        frame = cv2.resize(frame,(frame_width,frame_height))

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.2, minNeighbors = 5, minSize= (30,30))

    if len(faces) > 0:
        for x,y,w,h in faces:

            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,50,50),2)

            roi = gray[y:y+h,x:x+w]
            roi = cv2.resize(roi,(48,48))    
            roi = roi.astype("float")  / 255.0
            roi = tf.keras.preprocessing.image.img_to_array(roi)
            roi = np.expand_dims(roi , axis = 0)

            result = model.predict(roi)[0]
            print(result)

            if result is not None:
                result = np.argmax(result)
                cv2.putText(frame,expressions[result],(x,y-10),font,medium_font,(255,255,255),1, cv2.LINE_AA)
    
    cv2.putText(frame,f"FPS : {time.time()-t}",(20,frame_height-20),font,small_font,(250,250,250),0,cv2.LINE_AA)
    cv2_imshow(frame)
    video_output.write(frame)

print("Finish")
cap.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

----

In [39]:
directory = "/content/drive/MyDrive/Deep_Learning/Face_Recognation/Data/"
model = tf.keras.models.load_model(os.path.join(directory,"model_01_expressions.h5"))

cap = cv2.VideoCapture("/content/Material/Videos/video_teste06.MOV")
conn,frame = cap.read()
print("Before Resize : ",frame.shape[1],frame.shape[0])

resize = True
width_max = 600

if (resize and frame.shape[1] > width_max):
    proportionate = frame.shape[1] / frame.shape[0]
    frame_width = width_max
    frame_height = int(frame_width / proportionate)

else:
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

print("After Resize: ",frame_width,frame_height)
print("".center(100,"-"))

save_video_path = "/content/drive/MyDrive/Deep_Learning/Face_Recognation/Videos"
fourcc= cv2.VideoWriter_fourcc(*"mp4v")
fps = 24
video_output = cv2.VideoWriter(save_video_path,fourcc,fps,(frame_width,frame_height))

face_detection_path = "/content/Material/haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(face_detection_path)

small_font,medium_font = .4, .7
font = cv2.FONT_HERSHEY_SIMPLEX

expressions = ['Anger', 'Disgusted', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral']

unique_face = True

while (cv2.waitKey(1) < 0):
    conn, frame = cap.read()

    if not conn:
        break

    t = time.time()

    if resize:
        frame = cv2.resize(frame,(frame_width,frame_height))

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.2, minNeighbors = 5, minSize= (30,30))

    if len(faces) > 0:
        for x,y,w,h in faces:

            if unique_face and len(faces) > 1:
                #print(faces) --> [[201  95 149 149]]
                max_area_face = faces[0]
                for face in faces:
                    if face[2]*face[3] > max_area_face[2]*max_area_face[3]:
                        max_area_face = face

                face = max_area_face
                (x,y,w,h) = max_area_face

            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,50,50),2)

            roi = gray[y:y+h,x:x+w]
            roi = cv2.resize(roi,(48,48))
            roi = roi.astype("float") / 255.0
            roi = tf.keras.preprocessing.image.img_to_array(roi)
            roi = np.expand_dims(roi,axis = 0)

            result = model.predict(roi)[0]
            if result is not None:
                if unique_face:
                    for (index,(emotion,prob)) in enumerate(zip(expressions,result)):
                        text = "{}: {:.2f}%".format(emotion,prob*100)
                        bar = int(prob*150)
                        left_space = 7
                        if bar <=left_space:
                            bar = left_space+ 1 # Barın sola kaymasını engellemek için
                        cv2.rectangle(frame,(left_space,(index*18)+ 7),
                                      (bar,(index*18) + 18),(200,250,20),-1)
                        cv2.rectangle(frame,(left_space,(index*18)+ 7),
                                      (bar,(index*18) + 18),(0,0,0),1)
                        cv2.putText(frame,text,(15,(index*18)+ 15),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,0),1,cv2.LINE_AA)

                main_result = np.argmax(result)
                cv2.putText(frame,expressions[main_result],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1,cv2.LINE_AA)                
                        
            if unique_face and len(faces) > 1:
                break

    cv2.putText(frame,f"FPS : {time.time() - t}",(20,frame_height-20),cv2.FONT_HERSHEY_SIMPLEX,0.5,(250,250,250),0,cv2.LINE_AA)

    cv2_imshow(frame)
    video_output.write(frame)

print("Finish")
cap.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.